## 1. Multiprocessing Basics

In [1]:
import multiprocessing
import time
import os

print("=" * 50)
print("MULTIPROCESSING BASICS")
print("=" * 50)

def worker(name, value):
    print(f"[{name}] PID: {os.getpid()}, Value: {value}")
    time.sleep(2)
    print(f"[{name}] Done")

if __name__ == '__main__':
    processes = []
    
    for i in range(3):
        p = multiprocessing.Process(
            target=worker,
            args=(f"Process-{i}", i*10)
        )
        processes.append(p)
        p.start()
    
    print(f"Main PID: {os.getpid()}")
    
    for p in processes:
        p.join()
    
    print("All processes completed")

MULTIPROCESSING BASICS
Main PID: 14288
All processes completed


## 2. Inter-Process Communication

In [2]:
import multiprocessing
import time

print("\n" + "=" * 50)
print("INTER-PROCESS COMMUNICATION")
print("=" * 50)

# Using Queue for IPC
que = multiprocessing.Queue()

def producer_process(queue, count):
    for i in range(count):
        item = f"Item-{i}"
        queue.put(item)
        print(f"Produced: {item}")
        time.sleep(0.5)

def consumer_process(queue, count):
    for i in range(count):
        item = queue.get(timeout=5)
        print(f"Consumed: {item}")
        time.sleep(0.5)

if __name__ == '__main__':
    prod = multiprocessing.Process(target=producer_process, args=(que, 3))
    cons = multiprocessing.Process(target=consumer_process, args=(que, 3))
    
    prod.start()
    cons.start()
    
    prod.join()
    cons.join()
    
    print("IPC completed")


INTER-PROCESS COMMUNICATION
IPC completed


## 3. Shared Memory

import multiprocessing
import time

print("\n" + "=" * 50)
print("SHARED MEMORY")
print("=" * 50)

def modify_value(val, lock):
    with lock:
        # Accessing shared value
        val.value += 10
        print(f"Modified value: {val.value}")
        time.sleep(0.5)

if __name__ == '__main__':
    # Shared value (thread-safe)
    shared_val = multiprocessing.Value('i', 0)  # 'i' = integer
    lock = multiprocessing.Lock()
    
    processes = []
    for i in range(3):
        p = multiprocessing.Process(
            target=modify_value,
            args=(shared_val, lock)
        )
        processes.append(p)
        p.start()
    
    for p in processes:
        p.join()
    
    print(f"\nFinal value: {shared_val.value}")

## 4. CPU-Bound Task Speedup

In [ ]:
import multiprocessing
import time

print("\n" + "=" * 50)
print("CPU-BOUND PERFORMANCE")
print("=" * 50)

def cpu_bound_task(n):
    """CPU intensive calculation"""
    count = 0
    for i in range(n):
        count += i ** 2
    return count

# Sequential
print("Sequential execution:")
start = time.time()
for i in range(4):
    result = cpu_bound_task(10_000_000)
seq_time = time.time() - start
print(f"Time: {seq_time:.2f}s")

# Parallel with multiprocessing
if __name__ == '__main__':
    print("\nMultiprocessing execution:")
    start = time.time()
    
    with multiprocessing.Pool(processes=4) as pool:
        results = pool.map(cpu_bound_task, [10_000_000]*4)
    
    mp_time = time.time() - start
    print(f"Time: {mp_time:.2f}s")
    print(f"Speedup: {seq_time / mp_time:.1f}x")


CPU-BOUND PERFORMANCE
Sequential execution:
Time: 3.64s

Multiprocessing execution:


## 5. Process Pool

import multiprocessing
import time

print("\n" + "=" * 50)
print("PROCESS POOL")
print("=" * 50)

def square(x):
    return x ** 2

def cube(x):
    return x ** 3

if __name__ == '__main__':
    numbers = [1, 2, 3, 4, 5]
    
    # Using Pool
    with multiprocessing.Pool(processes=4) as pool:
        # map - blocks until all results ready
        squares = pool.map(square, numbers)
        print(f"Squares: {squares}")
        
        # imap - returns iterator
        cubes_iter = pool.imap(cube, numbers)
        print(f"Cubes: {list(cubes_iter)}")
        
        # apply_async - non-blocking
        result = pool.apply_async(square, (10,))
        print(f"10 squared: {result.get()}")

## 6. Custom Initialization

import multiprocessing

print("\n" + "=" * 50)
print("POOL WITH INITIALIZER")
print("=" * 50)

def init_worker():
    print(f"Worker initialized: PID {multiprocessing.current_process().pid}")

def process_data(x):
    return x * 2

if __name__ == '__main__':
    with multiprocessing.Pool(
        processes=2,
        initializer=init_worker
    ) as pool:
        results = pool.map(process_data, range(5))
        print(f"Results: {results}")

## 7. Comparison: Threading vs Multiprocessing

import threading
import multiprocessing
import time

print("\n" + "=" * 50)
print("THREADING vs MULTIPROCESSING")
print("=" * 50)

def cpu_task(n):
    count = 0
    for i in range(n):
        count += i ** 2
    return count

task_size = 50_000_000

# Threading (slow for CPU-bound)
print("Threading (CPU-bound):")
start = time.time()
threads = [threading.Thread(target=cpu_task, args=(task_size,)) for _ in range(4)]
for t in threads:
    t.start()
for t in threads:
    t.join()
threading_time = time.time() - start
print(f"Time: {threading_time:.2f}s")

# Multiprocessing (fast for CPU-bound)
if __name__ == '__main__':
    print("\nMultiprocessing (CPU-bound):")
    start = time.time()
    with multiprocessing.Pool(4) as pool:
        pool.map(cpu_task, [task_size]*4)
    mp_time = time.time() - start
    print(f"Time: {mp_time:.2f}s")
    print(f"\nMultiprocessing is {threading_time/mp_time:.1f}x faster for CPU-bound tasks")